So for part 2, we should also find the weight that each disk supports. So we'll want some new structures, let's use `dict`s, which implement the tree structures, and the weight distributions.

Let's start off by creating a tree that represents just the structure of the tower; we can worry about the weights after:

In [1]:
import re

Start with the example input again:

In [2]:
input_str='''pbga (66)
xhth (57)
ebii (61)
havc (66)
ktlj (57)
fwft (72) -> ktlj, cntj, xhth
qoyq (66)
padx (45) -> pbga, havc, qoyq
tknk (41) -> ugml, padx, fwft
jptl (61)
ugml (68) -> gyxo, ebii, jptl
gyxo (61)
cntj (57)'''

Let's go through each line: if it contains balancing information, add that to a set `supported_set`, and add any other cases to a set `disks_set`.

This time, rather than simply adding to a set `supported_set`, let's use a dict which implements the tree structure of the tower. I'm also changing the disks representation, so that we store the weight, and eventually, the total supported weight:

In [3]:
disks_dict={}

for nextLine_str in input_str.split('\n'):
    if '->' in nextLine_str:
        m_re=re.match('\s*(?P<name>\w+) \((?P<weight>\d+)\) ->(?P<rest>.*)', nextLine_str)
        
        disks_dict[m_re.group('name')]={'weight':int(m_re.group('weight')),
                                        'supporting':{disk.strip() for disk in m_re.group('rest').split(',')}}
    else:
        m_re=re.match('\s*(?P<name>\w+) \((?P<weight>\d+)\)', nextLine_str)
        
        disks_dict[m_re.group('name')]={'weight':int(m_re.group('weight'))}
print(disks_dict)


{'pbga': {'weight': 66}, 'havc': {'weight': 66}, 'cntj': {'weight': 57}, 'xhth': {'weight': 57}, 'padx': {'weight': 45, 'supporting': {'pbga', 'qoyq', 'havc'}}, 'fwft': {'weight': 72, 'supporting': {'cntj', 'ktlj', 'xhth'}}, 'ugml': {'weight': 68, 'supporting': {'gyxo', 'jptl', 'ebii'}}, 'qoyq': {'weight': 66}, 'ktlj': {'weight': 57}, 'tknk': {'weight': 41, 'supporting': {'ugml', 'fwft', 'padx'}}, 'gyxo': {'weight': 61}, 'jptl': {'weight': 61}, 'ebii': {'weight': 61}}


And now find the disk which isn't in one of the `supporting` sets:

In [4]:
nonRoot_set=set()
for vals in disks_dict.values():
    nonRoot_set=nonRoot_set.union(vals.get('supporting', {}))
nonRoot_set

rootDisk_str=(set(disk for disk in disks_dict)-nonRoot_set).pop()
rootDisk_str

'tknk'

Now build the total weight (own weight + supported weights) recursively:

In [5]:
def set_total_weights(startDisk_str, disksIn_dict):
    '''
    Find the total weight supported (including itself) by each
    disk in the tower. Update disksIn_ls with those weights.
    
    Returns the total weight on startDisk_str
    '''
    
    # If the disk is not supporting any other disks:
    if 'supporting' not in disksIn_dict[startDisk_str]:
        # Set the totalWeight to the disk's own weight, and
        # return that value
        disksIn_dict[startDisk_str]['totalWeight']=disksIn_dict[startDisk_str]['weight']
        return disksIn_dict[startDisk_str]['totalWeight']
    
    # Otherwise, build the total weights from the start
    # disk recursively:
    else:
        disksIn_dict[startDisk_str]['totalWeight'] = \
            sum([set_total_weights(d, disksIn_dict) 
                 for d in disksIn_dict[startDisk_str]['supporting']]) + \
            disksIn_dict[startDisk_str]['weight']
        return disksIn_dict[startDisk_str]['totalWeight']
        

print(set_total_weights('tknk', disks_dict))
print()
disks_dict


778



{'cntj': {'totalWeight': 57, 'weight': 57},
 'ebii': {'totalWeight': 61, 'weight': 61},
 'fwft': {'supporting': {'cntj', 'ktlj', 'xhth'},
  'totalWeight': 243,
  'weight': 72},
 'gyxo': {'totalWeight': 61, 'weight': 61},
 'havc': {'totalWeight': 66, 'weight': 66},
 'jptl': {'totalWeight': 61, 'weight': 61},
 'ktlj': {'totalWeight': 57, 'weight': 57},
 'padx': {'supporting': {'havc', 'pbga', 'qoyq'},
  'totalWeight': 243,
  'weight': 45},
 'pbga': {'totalWeight': 66, 'weight': 66},
 'qoyq': {'totalWeight': 66, 'weight': 66},
 'tknk': {'supporting': {'fwft', 'padx', 'ugml'},
  'totalWeight': 778,
  'weight': 41},
 'ugml': {'supporting': {'ebii', 'gyxo', 'jptl'},
  'totalWeight': 251,
  'weight': 68},
 'xhth': {'totalWeight': 57, 'weight': 57}}

So we seem to be building the tower correctly. Now we need to owrk out which is the incorrect weight.

In [7]:
disksIn_dict=disks_dict
startDisk_str='fwft'

[set_total_weights(d, disksIn_dict) for d in disksIn_dict[startDisk_str]['supporting']]

[57, 57, 57]

OK, now try with the test case:

In [8]:
with open('data/day7.txt') as fIn:
    input_str=fIn.read()

In [9]:
disks_set=set()
supported_set=set()

for nextLine_str in input_str.split('\n'):
    if '->' in nextLine_str:
        m_re=re.match('\s*(?P<name>\w+) \((?P<weight>\d+)\) ->(?P<rest>.*)', nextLine_str)
        
        disks_set.add((m_re.group('name'), m_re.group('weight')))
        supported_set=supported_set.union(set([disk.strip() for disk in m_re.group('rest').split(',')]))
    else:
        m_re=re.match('\s*(?P<name>\w+) \((?P<weight>\d+)\)', nextLine_str)
        if m_re:
            disks_set.add((m_re.group('name'), m_re.group('weight')))
        
baseDisk_str=({s[0] for s in disks_set}-supported_set).pop()
baseDisk_str

'hlhomy'